In [155]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from msedge.selenium_tools import Edge, EdgeOptions

import time
from random import *
from tqdm import tqdm

In [156]:
options = EdgeOptions()
options.use_chromium = True 
options.add_argument("headless")
options.add_argument("disable-gpu")

# Message: element not interactable 오류 방지
options.add_argument('--window-size=1920x1080')

driver = Edge('./msedgedriver.exe', options=options)

# 메뉴 주소 받아오기
- 가끔씩 올림픽 특수로 메뉴가 추가되고, bs4 크롤링을 막아놓았기 때문

In [157]:
base_url = "https://sports.news.naver.com/index.nhn"
driver.get(base_url)

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [158]:
# 메뉴 태그
menu_set = soup.select('a.link_main_menu')
time.sleep(uniform(0.3, 0.7))

menu_list = []
for menu in menu_set[1:-4]:
    link = menu.get('href').split('/')[1]
    menu_list.append(link)

In [159]:
for menu in menu_list:
    main_url = f"https://sports.news.naver.com/{menu}/news/index.nhn?isphoto=N"
    print(main_url)

# 드라이버 종료하고 새 드라이버
driver.close()

https://sports.news.naver.com/kbaseball/news/index.nhn?isphoto=N
https://sports.news.naver.com/wbaseball/news/index.nhn?isphoto=N
https://sports.news.naver.com/kfootball/news/index.nhn?isphoto=N
https://sports.news.naver.com/wfootball/news/index.nhn?isphoto=N
https://sports.news.naver.com/basketball/news/index.nhn?isphoto=N
https://sports.news.naver.com/volleyball/news/index.nhn?isphoto=N
https://sports.news.naver.com/golf/news/index.nhn?isphoto=N
https://sports.news.naver.com/general/news/index.nhn?isphoto=N


# 네이버 스포츠 특징
- 총 몇 페이지가 있는지 알려주는 인디케이터가 없다. <br>
=> 1~10페이지까지 돈 다음에 마지막 페이지에서 selenium으로 일일히 '다음' 버튼을 눌러줘야 함 <br>
=> 
- 최대 페이지를 넘어가면 마지막 페이지가 똑같이 나온다. <br>
=> page_num += 1 으로 while문을 돌리는데, 마지막 페이지를 찾아서 while문을 끝내야 한다.

## Pagination 분석
- 현재 페이지의 Tag는 <strong\>이다.
### 언제 try-except로 다음 버튼을 눌러야 할까? 
- %10 == 0 일때 다음 버튼을 찾고 누른다.
- 없으면 알아서 10이 오기 전에 (딱 10에 끝나도 상관없다) Last number가 나와 끝난다.

In [213]:
driver = Edge('./msedgedriver.exe', options=options)

In [214]:
driver.get(tmp)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

### 특히 headless일땐 빠르기 때문에 select 메소드가 태그를 찾지 못하고 넘어가는 경우가 있기 때문에 select 코드 다음에 sleep 코드를 넣어줘야 한다.

In [215]:
# 모든 메뉴를 돌아가며 크롤링
for menu in tqdm(menu_list):
    time.sleep(uniform(0.3, 0.7))

    page_num = 1
    while True:
        url = f"https://sports.news.naver.com/{menu}/news/index.nhn?isphoto=N&page={page_num}"
        driver.get(url)
        time.sleep(uniform(0.3, 0.7))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 새로운 메뉴마다 driver.get(url)로 새로운 창을 띄우지 않기 위해 기존 창으로 돌아가기
        driver.switch_to.window(driver.window_handles[0])
        time.sleep(uniform(0.3, 0.7))


        titles = soup.select('a.title')
        today = datetime.now().strftime("%Y_%m_%d")
        f = open(f'./result/{today} 네이버스포츠 기사제목.txt', 'a', encoding='utf8')
        for title in titles:
            f.write(title.text + '\n')
        f.close()    

        
        if page_num % 10 == 0:
        # 페이지가 10, 20, 30 일때 '다음' 버튼이 있으면 누르고 없으면 pass한다.
            try:
                # page_source를 위에서 사용했어도 여전히 driver method 사용 가능
                next = driver.find_element_by_css_selector('#_pageList > a.next')
                time.sleep(uniform(0.3, 0.7))
                next.click()
            # '다음' 버튼이 없을 때    
            except:
                """ 언제 끝내야 하는가?
                페이지네이션의 마지막 페이지 넘버를 받아온다.
                그게 현재 페이지랑 같다면 루프를 닫으면 된다.

                주의: 현재 페이지의 태그는 strong이기 때문에 10배수 페이지에서 받아올 수 없다.
                (10배수 페이지 == 현재 페이지, 그러므로 마지막 태그가 10이 아닌 9가 됨) 
                """
                pass

        # 그러므로 마지막 페이저 넘버를 시작할 때, 즉 페이지넘버 = 1일때 받아야 함
        if page_num % 10 == 1:
        # '다음' 버튼이 있기 때문에 pagebox[-1]가 마지막 페이지의 인덱스다.  
            try:
                pagebox = soup.select('#_pageList a')
                time.sleep(uniform(0.3, 0.7))
                
                # 페이지가 2개라서 a태그가 하나밖에 없을 때
                if len(pagebox) == 1:
                    last_page = int(pagebox[0].text) 
                    # 여기원래 int(pagebox[0].text) + 1 이었음. 안되면 이걸로
                # 페이지 3개 이상
                else:
                    last_page = int(pagebox[-1].text)
                    # 여기원래 int(pagebox[-2].text) + 1 이었음. 안되면 이걸로
            except:
                # 페이지가 1개라서(ex.마지막 페이지가 21) a는 하나도 없고 자기자신이 strong일 때
                last_page = int(soup.select_one('#_pageList strong').text)

        if last_page == page_num:
            break
        else:
            page_num += 1

100%|██████████| 8/8 [00:58<00:00,  7.26s/it]


In [216]:
pagebox = soup.select('#_pageList')
# 여기는 '다음'이란게 있어서 실제 사용할때는 -1이 되어야 함
pagebox

[<div class="paginate" id="_pageList" style="">
 <a data-id="1" href="#" onclick="clickcr(this, 'nwl.paging', '', '', event);">1</a>
 <a data-id="2" href="#" onclick="clickcr(this, 'nwl.paging', '', '', event);">2</a>
 <strong>3</strong>
 </div>]

In [217]:
pagebox = soup.select('#_pageList a')
int(pagebox[-2].text)

1